In [2]:
import ee
import numpy as np
import pandas as pd

from earthshot import mon_stats
from earthshot import water_viz as vis
from earthshot import normalize as norm
from earthshot import water_common as common
from statistics import mean

import geemap.eefolium as geemap
import folium
from folium import plugins

import matplotlib.pyplot as plt
%matplotlib inline


Loading BokehJS ...

In [3]:
ee.Initialize()

In [23]:
bbox_global = ee.Geometry.BBox(-180,-90,180,90)

In [28]:
#Importing runoff in. Data is in average hourly runoff on a monthly basis, so convert to annual by multiplying by 24*30= 720
#Resolution = 0.1 arc degrees, ~11 km. units = m/ year
runoff_clim = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY").select('surface_runoff')
runoff_img = runoff_clim.mean().multiply(12)
runoff_img = norm.img_scale(runoff_img, area_of_interest = bbox_global)

evap_clim = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY").select('evaporation_from_bare_soil').mean().multiply(12)
evap_clim_2 = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY").select('evaporation_from_open_water_surfaces_excluding_oceans').mean().multiply(12)
evap_img = evap_clim.add(evap_clim_2)

In [5]:
#Import precip in. Data is average monthly precip, so sum up over months to obtain annual precip
#https://developers.google.com/earth-engine/datasets/catalog/OpenLandMap_CLM_CLM_PRECIPITATION_SM2RAIN_M_v01#bands
#Scale = 1 km, units = mm/ year

precip_clim = ee.Image("OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01")
precip_img = precip_clim.reduce('sum').multiply(720)


In [6]:
#Import evapotranspiration in. Data is in mm/ day, averaged over a dekad (10 days). Multiply by days/ year to get average annual ET.
#Scale = 0.00223 arc degrees, ~247 m at equator. Units = mm/ year

et_clim = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").select('aet')
et_img = et_clim.mean().multiply(12)



In [26]:
water_content = ee.Image("OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01").select('b0')
water_content_scaled = norm.img_scale(water_content, area_of_interest = bbox_global)


In [29]:
print(norm.img_range(runoff_img))

[0, 1]


In [27]:
print(norm.img_range(water_content_scaled))

[0, 1]


In [33]:
infiltration_img = runoff_img.subtract(water_content_scaled)

In [34]:
print(norm.img_range(infiltration_img))

[-0.9644854456624831, 0.51]


In [38]:
the_map = geemap.Map()

vis_params = {
    'min': -0.5, 'max': 0.25, 'dimensions': 512,
    'palette': ['Red', 'Yellow', 'Blue']}

the_map.addLayer(infiltration_img, vis_params, name = 'Infiltration score')
vis.folium_display(the_map)